In [1]:
# place to mess with cantera mechanism and see how it changes the ignition delay

In [2]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane'
cti_path = os.path.join(basedir, 'chem_annotated.cti')
gas = ct.Solution(cti_path)

In [4]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)

# slice just table 7, where phi=1.0
table7 = df_exp[df_exp['Table'] == 7]
# Define Initial conditions using experimental data
tau7 = table7['time (ms)'].values.astype(float)  # ignition delay
T7 = table7['T_C'].values  # Temperatures
P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm


# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
# for phi = 1
x_diluent = 0.7649
conc_dict = {
    'O2(2)': 0.2038,
    'butane(1)': 0.03135
}

for i in range(0, len(table7)):
    x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    concentrations.append(conc_dict)


In [5]:
def perturb_reaction_ct(rxn, delta):
    # should just be Arrhenius types
    new_kinetics = ct._cantera.Arrhenius(
        np.exp(delta) * rxn.rate.pre_exponential_factor, 
        rxn.rate.temperature_exponent,
        rxn.rate.activation_energy
    )
    rxn.rate = new_kinetics


In [6]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T, P, X):
    # function to run a RCM simulation

    t_end = 1.0  # time in seconds
    gas.TPX = T, P, X

    env = ct.Reservoir(ct.Solution('air.yaml'))
    # env = ct.Reservoir(ct.Solution('air.xml'))
    reactor = ct.IdealGasReactor(gas)
    wall = ct.Wall(reactor, env, A=1.0, velocity=0)
    reactor_net = ct.ReactorNet([reactor])
    # # allegedly faster solving
    # reactor_net.derivative_settings = {"skip-third-bodies": True, "skip-falloff": True}
    # reactor_net.preconditioner = ct.AdaptivePreconditioner()

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        reactor_net.step()

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    i = np.argmax(slopes)
    return times[i]

In [13]:
base_delay = 0.14857534592261107

In [20]:
new_kinetics = ct._cantera.Arrhenius(1.0, 2.0, 3.0)

In [22]:
rxn2 = gas.reactions()[0]

In [24]:
rxn2.rate = ct._cantera.Arrhenius(1.0, 2.0, 3.0)

In [25]:
gas.modify_reaction(0, rxn2)

In [28]:
gas.reactions().index(rxn2)

ValueError: <ElementaryReaction: H(14) + O2(2) <=> O(5) + OH(15)> is not in list

In [27]:
gas.reactions()[0].rate

Arrhenius(A=1, b=2, E=3)

In [16]:
dir(gas)

['DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'ID',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__composition_to_array',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_init_yaml',
 '_native_state',
 '_partial_states',
 '_references',
 'activities',
 'activity_coefficients',
 'add_reaction',
 'add_species',
 'add_species_alias',
 'atomic_weight',
 'atomic_weights',
 'basis',

In [32]:
i = 7

for reaction_number in 

gas = ct.Solution(cti_path)
delta = 1.0
if type(gas.reactions()[reaction_number].rate) == ct._cantera.Arrhenius:
    print(f"perturbing reaction {gas.reactions()[reaction_number]}")
    
    # create the new reaction
    new_reaction = gas.reactions()[reaction_number]
    new_kinetics = ct._cantera.Arrhenius(
        np.exp(delta) * new_reaction.rate.pre_exponential_factor, 
        new_reaction.rate.temperature_exponent,
        new_reaction.rate.activation_energy
    )
    new_reaction.rate = new_kinetics
    gas.modify_reaction(reaction_number, new_reaction)
    

# run the simulations at condition #7
i = 7
X = concentrations[i]
delay_time = get_delay(gas, T7[i], P7[i], X)

print(delay_time)
print(f'Delta={base_delay - delay_time}')


perturbing reaction H(14) + O2(2) <=> O(5) + OH(15)
0.07807946164848509
Delta=0.07049588427412598
